In [ ]:
%load_ext autoreload
%autoreload 2
import tensorboard
import ioutils 
from esm.models.esm3 import ESM3
import os 
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
os.environ["TOKENIZERS_PARALLELISM"]="true"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
from scipy import stats
nonan = df1[~pd.isna(df1["Prediction.1"])]
res = stats.spearmanr(nonan["Prediction.1"][:-1], nonan["2e21"][:-1])
res

SignificanceResult(statistic=0.020449634018821886, pvalue=0.8031850419572404)

In [4]:
# df1.columns = ["a", "b", "c", "d", "e"]
df1 = df1[~pd.isna(df1["translation"])]
df1

,species_name,seq_id,taxid,Name,gene,locus_tag,codon_start,product,protein_id,translation,is_homo,is_complete,num_tmrs,fasta_file_path
0,Acidianus filamentous virus 2,NC_009884,300186,"Acidianus filamentous virus 2, complete genome",NaN,AFV2_gp01,1,hypothetical protein,YP_001496926.1,MKKVGEEEIKQEENEKEKIVKKLNESDVKSIIEDVLKNKEKYGLEA...,False,True,nonhuman_or_incomplete,./fasta/task_1\Acidianus_filamentous_virus_2\N...
1,Acidianus filamentous virus 2,NC_009884,300186,"Acidianus filamentous virus 2, complete genome",NaN,AFV2_gp02,1,hypothetical protein,YP_001496927.1,MKKMKGEKKAKREGYILLQEIPKLEYIFKNWNEFEDNFYVAEYNYN...,False,True,nonhuman_or_incomplete,./fasta/task_1\Acidianus_filamentous_virus_2\N...
2,Acidianus filamentous virus 2,NC_009884,300186,"Acidianus filamentous virus 2, complete genome",NaN,AFV2_gp03,1,hypothetical protein,YP_001496928.1,MQVLQIVKEKAKMRQNMNFFETTIYEKYDLRDKIKPGQKFVVRQGD...,False,True,nonhuman_or_incomplete,./fasta/task_1\Acidianus_filamentous_virus_2\N...
3,Acidianus filamentous virus 2,NC_009884,300186,"Acidianus filamentous virus 2, complete genome",NaN,AFV2_gp04,1,transcriptional regulator,YP_001496929.1,MIITIEEEVYNNLKKRAEKEEISIPAIVRKFIVSYFNLEDSTKDYK...,False,True,nonhuman_or_incomplete,./fasta/task_1\Acidianus_filamentous_virus_2\N...
4,Acidianus filamentous virus 2,NC_009884,300186,"Acidianus filamentous virus 2, complete genome",NaN,AFV2_gp05,1,hypothetical protein,YP_001496930.1,MGRKKMMEKEKIEEMIKEINEELDKLCLFPLEFVDGGWRYYVHANA...,False,True,nonhuman_or_incomplete,./fasta/task_1\Acidianus_filamentous_virus_2\N...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227179,Vesicular exanthema of swine virus,U76874,35612,"Vesicular exanthema of swine virus stain A48, ...",NaN,NaN,1,capsid protein precursor,AAC13889.2,MATTHTLLSFDNLEFLLHRKDLTDLYGKRCGTLNLVINPYELFLPD...,True,True,NaN,./fasta/task_9/Vesicular_exanthema_of_swine_vi...
227180,Vesicular exanthema of swine virus,U76874,35612,"Vesicular exanthema of swine virus stain A48, ...",NaN,NaN,1,small basic protein,AAC13890.1,MNYANFGLDFLNSVANAAVEGKKLDLASRGLQLRSRALDTERDFNY...,True,True,NaN,./fasta/task_9/Vesicular_exanthema_of_swine_vi...
227181,Vesicular exanthema of swine virus,U15301,35612,"San Miguel sea lion virus serotype 1, complete...",NaN,NaN,1,non-structural polyprotein,AAA96501.2,MAQTLSKISNKENASVGLWPKRFKPHQPTPTWMVRCGPLDHDSRHG...,True,True,NaN,./fasta/task_9/Vesicular_exanthema_of_swine_vi...
227182,Vesicular exanthema of swine virus,U15301,35612,"San Miguel sea lion virus serotype 1, complete...",NaN,NaN,1,capsid protein precursor,AAG13639.1,MATTHTLLSFDDLEFLLHKKDLTDLYGERCGTLNLVINPYDLFLPD...,True,True,NaN,./fasta/task_9/Vesicular_exanthema_of_swine_vi...


In [5]:
f = open("/home/tyfei/datasets/ion_channel/Interprot/cds_data_all.fasta", "w")
cnt = 0
for i in df1["translation"]:
    # print(i)
    f.write(">"+str(cnt)+"\n")
    cnt += 1  
    f.write(i+"\n")
f.close()

In [3]:
model = ESM3.from_pretrained("esm3_sm_open_v1")

In [3]:
# seqs = ioutils.readFasta("/home/tyfei/datasets/ion_channel/Interprot/Negative_sample/virus_proteins_TMHMM_keywords.fasta")
seqs = ioutils.readFasta("/home/tyfei/datasets/ion_channel/Interprot/ion_channel/mmseqs_kingdom/Eukaryota.fasta")
seqss = [i for i in seqs]

In [4]:
len(seqss[1024][1])
allres = []

In [11]:
# torch.manual_seed(1509)
with torch.no_grad():

    for steps, seqidx in zip((16, 16, 16, 16, 16, 16), (4, 98, 23, 4, 98, 23)):
        model = model.to("cuda:7")
        protein = ESMProtein(sequence=seqss[seqidx][1])
        protein = model.generate(
            protein,
            GenerationConfig(track="secondary_structure", num_steps=steps),
        )
        protein = model.generate(
            protein, GenerationConfig(track="sasa", num_steps=steps)
        )
        protein = model.generate(
            protein, GenerationConfig(track="structure", num_steps=steps)
        )
        res = model.encode(protein)
        res = model(sequence_tokens=res.sequence.unsqueeze(0), 
                    structure_tokens=res.structure.unsqueeze(0), 
                    ss8_tokens = res.secondary_structure.unsqueeze(0), 
                    sasa_tokens=res.sasa.unsqueeze(0)
                    ) 

        res = res.embeddings
        res = res.cpu().squeeze()
        allres.append(res.mean(axis=0).numpy())
# res.mean(axis=0)
# res1 = res.mean(axis=0)

In [15]:
def dist(x, y):
    t = np.sum(((x - y) ** 2))
    t = np.sqrt(t)
    # t = np.sum(x*y)
    # t = t/(np.sqrt(np.sum(x**2))*np.sqrt(np.sum(y**2)))
    return t 

dist(allres[2], allres[4])
    

3363.9614

In [65]:
alldists = [] 
for i in allres:
    for j in allres:
        alldists.append(dist(i, j))
np.mean(alldists)

456.09647

In [38]:
alldists = [] 
for i in allres:
    for j in allres:
        alldists.append(dist(i, j))
np.mean(alldists)

346.19003

In [35]:
alldists = [] 
for i in allres:
    for j in allres:
        alldists.append(dist(i, j))
np.mean(alldists)

1090.0117

In [34]:
dist(allres[3], allres[1])

985.5462

In [14]:
torch.manual_seed(1509)
with torch.no_grad():
    model = model.to("cuda:2")
    protein = ESMProtein(sequence=seqss[1][1])
    protein = model.generate(
        protein,
        GenerationConfig(track="secondary_structure", num_steps=64),
    )
    protein = model.generate(
        protein, GenerationConfig(track="sasa", num_steps=64)
    )
    protein = model.generate(
        protein, GenerationConfig(track="structure", num_steps=64)
    )
    res = model.encode(protein)
    res = model(sequence_tokens=res.sequence.unsqueeze(0), 
                structure_tokens=res.structure.unsqueeze(0), 
                # ss8_tokens = res.secondary_structure.unsqueeze(0), 
                # sasa_tokens=res.sasa.unsqueeze(0)
                ) 

    res = res.embeddings
    res = res.cpu().squeeze()
res.mean(axis=0)

tensor([  -2.6954, -124.1741,   72.6040,  ...,  -25.1897,  -32.3494,
          16.3941])

In [6]:
res

ESMProteinTensor(sequence=tensor([ 0, 20,  8, 12, 10, 12, 15, 12, 13, 15,  4, 10, 16, 12,  7,  5, 19, 18,
         8,  9, 18,  8,  9,  9,  7,  8, 12, 17,  7, 13,  4, 11, 13,  9,  4, 20,
        19, 12, 18,  5,  5,  4,  6,  6,  8,  7, 17, 12, 22,  5, 12, 12, 14,  4,
         8,  5,  8,  7, 18, 19, 10,  6,  5,  9, 17, 12,  7, 18, 17,  4, 14,  7,
         8, 15,  7, 15,  8, 23,  4, 23,  8, 18, 21, 17, 13,  5, 12, 12, 17, 12,
         9, 14, 13,  4,  9, 17, 17,  4,  7, 15,  4,  8,  8, 19, 21,  7,  7,  8,
         7, 13, 23, 17, 15,  9,  4, 20, 14, 12, 10, 11, 13, 11, 11, 12, 23,  4,
         8, 12, 13, 16, 15, 15,  8, 19,  7, 18, 17, 18, 21, 15, 19,  9,  9, 15,
        23, 23,  6, 10, 11,  7, 12, 21,  4,  9, 22,  4,  4,  6, 18, 12, 15, 23,
        12,  8, 16, 21, 16, 21,  4,  5, 12, 20, 18, 15, 13, 13, 17, 12, 12, 20,
        15, 11, 14,  6, 17, 11, 13,  5, 18,  8, 10,  9, 19,  8, 20, 11,  9, 23,
         8, 16,  9,  4, 16, 15, 18,  8, 18, 15, 12,  5, 12,  8,  8,  4, 17, 15,
         4, 10

In [3]:
from pytorch_lightning.loggers import TensorBoardLogger 
logger = TensorBoardLogger("tb_logs", name="hello") 
logger.log_hyperparams({"lambda":0.1, "path":"/home/tyfei/ionChannel/ckptsbaseline/test4"})
logger.log_hyperparams({"lambda":0.1, "steps":1500})
logger.log_hyperparams({"lambda":0.15, "steps":3000})

In [4]:
logger.log_hyperparams({"lambda":0.1, "path":"/home/tyfei/ionChannel/ckptsbaseline/test4"})
logger.log_hyperparams({"lambda":0.1, "steps":1500})
logger.log_hyperparams({"lambda":0.15, "steps":3000})

In [2]:
import json
import trainUtils 
path = "/home/tyfei/ionChannel/ckptsesm3/Lora3continue/"
with open(os.path.join(path, "config.json"), "r") as f:
    configs = json.load(f)
ds = trainUtils.loadDataset(configs)

In [3]:
pretrainmodel = trainUtils.loadPretrainModel(configs=configs)
pretrainmodel

ESM3(
  (encoder): EncodeInputs(
    (sequence_embed): Embedding(64, 1536)
    (plddt_projection): Linear(in_features=16, out_features=1536, bias=True)
    (structure_per_res_plddt_projection): Linear(in_features=16, out_features=1536, bias=True)
    (structure_tokens_embed): Embedding(4101, 1536)
    (ss8_embed): Embedding(11, 1536)
    (sasa_embed): Embedding(19, 1536)
    (function_embed): ModuleList(
      (0-7): 8 x Embedding(260, 192, padding_idx=0)
    )
    (residue_embed): EmbeddingBag(1478, 1536, mode='sum', padding_idx=0)
  )
  (transformer): TransformerStack(
    (blocks): ModuleList(
      (0): UnifiedTransformerBlock(
        (attn): MultiHeadAttention(
          (layernorm_qkv): Sequential(
            (0): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=1536, out_features=4608, bias=False)
          )
          (out_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (q_ln): LayerNorm((1536,), eps=1e-05, el

In [4]:
import models
model = models.IonclfESM3.load_from_checkpoint("/home/tyfei/ionChannel/ckptsesm3/Lora3/epoch=5-validate_acc=0.95.ckpt", esm_model = pretrainmodel)

In [6]:
a = torch.load("/home/tyfei/ionChannel/ckptsesm3/Lora3/epoch=5-validate_acc=0.95.ckpt")
a.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers'])

In [7]:
a["global_step"]

5478

In [27]:
for i, j in model.named_parameters():
    if j.requires_grad:
        print(i)

esm_model.transformer.blocks.43.attn.layernorm_qkv.1.lora.A
esm_model.transformer.blocks.43.attn.layernorm_qkv.1.lora.B
esm_model.transformer.blocks.43.attn.out_proj.lora.A
esm_model.transformer.blocks.43.attn.out_proj.lora.B
esm_model.transformer.blocks.43.ffn.1.lora.A
esm_model.transformer.blocks.43.ffn.1.lora.B
esm_model.transformer.blocks.43.ffn.3.lora.A
esm_model.transformer.blocks.43.ffn.3.lora.B
esm_model.transformer.blocks.44.attn.layernorm_qkv.1.lora.A
esm_model.transformer.blocks.44.attn.layernorm_qkv.1.lora.B
esm_model.transformer.blocks.44.attn.out_proj.lora.A
esm_model.transformer.blocks.44.attn.out_proj.lora.B
esm_model.transformer.blocks.44.ffn.1.lora.A
esm_model.transformer.blocks.44.ffn.1.lora.B
esm_model.transformer.blocks.44.ffn.3.lora.A
esm_model.transformer.blocks.44.ffn.3.lora.B
esm_model.transformer.blocks.45.attn.layernorm_qkv.1.lora.A
esm_model.transformer.blocks.45.attn.layernorm_qkv.1.lora.B
esm_model.transformer.blocks.45.attn.out_proj.lora.A
esm_model.trans

In [20]:
q = model.configure_optimizers()
q

get training optimizer


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0005
    maximize: False
    weight_decay: 0.005
)

In [ ]:
q.param_groups

In [ ]:
q.state_dict()

In [30]:
len(a["optimizer_states"][0]["param_groups"])

6

In [ ]:
q.load_state_dict(a["optimizer_states"][0])

In [14]:
import pytorch_lightning
import pytorch_lightning.trainer 
trainer = pytorch_lightning.Trainer(devices=7, max_epochs=24, accumulate_grad_batches=16, accelerator="gpu", log_every_n_steps=1)
trainer.fit(model, ds, ckpt_path="/home/tyfei/ionChannel/ckptsesm3/Lora3/epoch=5-validate_acc=0.95.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [20]:
import pickle

"/home/tyfei/datasets/ion_channel/Interprot/Negative_sample/refseq_rmdup_mmseq_noTMHMM2.pkl",
"/home/tyfei/datasets/ion_channel/Interprot/Negative_sample/refseq_rmdup_mmseq_noTMHMM.pkl",
"/home/tyfei/datasets/ion_channel/Interprot/Negative_sample/refseq_rmdup_mmseq_TMHMM_keywords2.pkl",
"/home/tyfei/datasets/ion_channel/Interprot/Negative_sample/refseq_rmdup_mmseq_TMHMM_keywords.pkl",
"/home/tyfei/datasets/ion_channel/Interprot/Negative_sample/virus_proteins_rmdup_clean_X0.1_TMHMM_mmseq_keywords_norefseq.pkl",
"/home/tyfei/datasets/ion_channel/Interprot/Negative_sample/virus_proteins_TMHMM_keywords2.pkl",
"/home/tyfei/datasets/ion_channel/Interprot/Negative_sample/interpro_neg2.pkl",
"/home/tyfei/datasets/ion_channel/Interprot/Negative_sample/interpro_neg.pkl"

            
with open("/home/tyfei/datasets/ion_channel/Interprot/test885.pkl", "rb") as f:
    t = pickle.load(f) 
len(t) 

885

In [44]:
for i in range(len(t)):
    sample = t[i]
    if str(i) != sample["id"]:
        print(i)
        break

In [45]:
t[-1]

{'randomseed': 1509,
 'model': 'esm3_sm_open_v1',
 'id': '883',
 'ori_seq': 'MDSTIRLVATIFLISLTQQIEVCNKAQQQGPYTLVDYQEKPLNISRIQIKVVKTSVATKGLNFHIGYRAVWRSYCYNGGSLDKNTGCYNDLIPKSPTESELRTWSKSQKCCTGPDAVDAWGSDARICWAEWKMELCHTAKELKKYSNNNHFAYHTCNLSWRCGLKSTHIEVRLQASGGLVSMVAVMPNGTLIPIEGTRPTYWTEDSFAYLYDPAGTEKKTESTFLWCFKEHIRPTTELSGAVYDTHYLGGTYDKNPQFNYYCRDNGYYFELPANRLVCLPTSCYKREGAIVNTMHPDTWKVSEKLHSASQFDVNNVVHSLVYETEGLRLALSQLDHRFATLSRLFNRLTQSLAKIDDRLLGTLLGQDVSSKFISPTKFMLSPCLSTPEGDSNCHNHSIYRDGRWVHNSDPTQCFSLSKSQPVDLYSFKELWLPQLLDVNVEGVVADEEGWSFVAQSKQALIDTMTYTKNGGKGTSLEDVLGYPSGWINGKLQGLLLNGAISWVVVIGVVLVGVCLMRRVF',
 'steps': 16,
 'seq_t': array([ 0, 20, 13,  8, 11, 12, 10,  4,  7,  5, 11, 12, 18,  4, 12,  8,  4,
        11, 16, 16, 12,  9,  7, 23, 17, 15,  5, 16, 16, 16,  6, 14, 19, 11,
         4,  7, 13, 19, 16,  9, 15, 14,  4, 17, 12,  8, 10, 12, 16, 12, 15,
         7,  7, 15, 11,  8,  7,  5, 11, 15,  6,  4, 17, 18, 21, 12,  6, 19,
        10,  5,  7, 22, 10,  8, 19, 23, 19, 17,  6,  6,  8,  4, 13, 15, 1

In [3]:
t[1].keys()

dict_keys(['randomseed', 'model', 'id', 'ori_seq', 'seq_t', 'structure_t', 'second_t', 'sasa_t', 'coordinates'])

In [11]:
maxlen = 0 
for i in t:
    if len(i["ori_seq"]) > maxlen:
        maxlen = len(i["ori_seq"]) 
maxlen

1500

In [53]:
t = ds.train_dataloader()#.dataset
len(t)

get train loader


1053

In [56]:
torch.stack([torch.randn(4, 1)]).mean()

tensor(0.4366)

In [54]:
from tqdm import tqdm
q = []
pbar = tqdm(t)
for i, (x1, l, x2) in enumerate(pbar):
    # print(i, l)
    # break
    q.append(l.item())
q

  0%|          | 0/1053 [00:00<?, ?it/s]

now epoch  1


100%|██████████| 1053/1053 [00:17<00:00, 61.38it/s] 


[1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,


In [45]:
len(q), sum(q)*2

(510, 420)

In [46]:
t.dataset.newEpoch()
res = t.dataset.getOrigin()

In [47]:
np.unique(res, return_counts=True)

(array(['Archaea.pkl', 'Viruses.pkl', 'random.pkl',
        'refseq_rmdup_mmseq_noTMHMM.pkl'], dtype='<U30'),
 array([ 10, 200, 200, 100]))

In [37]:
t.dataset.getOrigin()

['Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Viruses.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Viruses.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Viruses.pkl',
 'Viruses.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Viruses.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Viruses.pkl',
 'Viruses.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Viruses.pkl',
 'Archaea.pkl',
 'Viruses.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Viruses.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Viruses.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archaea.pkl',
 'Archae

In [2]:
import os 
os.path.basename("/home/tyfei/datasets/ion_channel/Interprot/Negative_sample/TEST_refseq_rmdup_mmseq_TMHMM.fasta")

'TEST_refseq_rmdup_mmseq_TMHMM.fasta'

In [ ]:
torch.utils.data.random_split()

In [2]:
from pytorch_lightning import trainer

In [3]:
lens = [] 
for i in seqss:
    lens.append(len(i[1]))

In [4]:

model = ESM3.from_pretrained("esm3_sm_open_v1")
# model = model.cuda(5)
import models 
newmodel = models.addlora(model, [45, 46, 47], [8, 8, 8], [16, 16, 16])
# newmodel = models.IonclfESM3(model).cuda(6)
newmodel.cuda(7)

ESM3(
  (encoder): EncodeInputs(
    (sequence_embed): Embedding(64, 1536)
    (plddt_projection): Linear(in_features=16, out_features=1536, bias=True)
    (structure_per_res_plddt_projection): Linear(in_features=16, out_features=1536, bias=True)
    (structure_tokens_embed): Embedding(4101, 1536)
    (ss8_embed): Embedding(11, 1536)
    (sasa_embed): Embedding(19, 1536)
    (function_embed): ModuleList(
      (0-7): 8 x Embedding(260, 192, padding_idx=0)
    )
    (residue_embed): EmbeddingBag(1478, 1536, mode='sum', padding_idx=0)
  )
  (transformer): TransformerStack(
    (blocks): ModuleList(
      (0): UnifiedTransformerBlock(
        (attn): MultiHeadAttention(
          (layernorm_qkv): Sequential(
            (0): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=1536, out_features=4608, bias=False)
          )
          (out_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (q_ln): LayerNorm((1536,), eps=1e-05, el

In [9]:
for i, j in newmodel.named_modules():
    print(i)


encoder
encoder.sequence_embed
encoder.plddt_projection
encoder.structure_per_res_plddt_projection
encoder.structure_tokens_embed
encoder.ss8_embed
encoder.sasa_embed
encoder.function_embed
encoder.function_embed.0
encoder.function_embed.1
encoder.function_embed.2
encoder.function_embed.3
encoder.function_embed.4
encoder.function_embed.5
encoder.function_embed.6
encoder.function_embed.7
encoder.residue_embed
transformer
transformer.blocks
transformer.blocks.0
transformer.blocks.0.attn
transformer.blocks.0.attn.layernorm_qkv
transformer.blocks.0.attn.layernorm_qkv.0
transformer.blocks.0.attn.layernorm_qkv.1
transformer.blocks.0.attn.out_proj
transformer.blocks.0.attn.q_ln
transformer.blocks.0.attn.k_ln
transformer.blocks.0.attn.rotary
transformer.blocks.0.geom_attn
transformer.blocks.0.geom_attn.s_norm
transformer.blocks.0.geom_attn.proj
transformer.blocks.0.geom_attn.out_proj
transformer.blocks.0.ffn
transformer.blocks.0.ffn.0
transformer.blocks.0.ffn.1
transformer.blocks.0.ffn.2
tran

In [8]:
for i, j in newmodel.named_modules():
    print(i)


encoder
encoder.sequence_embed
encoder.plddt_projection
encoder.structure_per_res_plddt_projection
encoder.structure_tokens_embed
encoder.ss8_embed
encoder.sasa_embed
encoder.function_embed
encoder.function_embed.0
encoder.function_embed.1
encoder.function_embed.2
encoder.function_embed.3
encoder.function_embed.4
encoder.function_embed.5
encoder.function_embed.6
encoder.function_embed.7
encoder.residue_embed
transformer
transformer.blocks
transformer.blocks.0
transformer.blocks.0.attn
transformer.blocks.0.attn.layernorm_qkv
transformer.blocks.0.attn.layernorm_qkv.0
transformer.blocks.0.attn.layernorm_qkv.1
transformer.blocks.0.attn.out_proj
transformer.blocks.0.attn.q_ln
transformer.blocks.0.attn.k_ln
transformer.blocks.0.attn.rotary
transformer.blocks.0.geom_attn
transformer.blocks.0.geom_attn.s_norm
transformer.blocks.0.geom_attn.proj
transformer.blocks.0.geom_attn.out_proj
transformer.blocks.0.ffn
transformer.blocks.0.ffn.0
transformer.blocks.0.ffn.1
transformer.blocks.0.ffn.2
tran

In [4]:
protein = ESMProtein(
    sequence="MGAALTLLGDLIATVSEAAAATGFSVAEIAAGEAAAAIEVQLASVATVEGLTTSEAIAAI\
GLIPQAYAVISGAPAAIAGFAALLQTVTGVSAVAQVGYRFFSDWDHKVSTVGLYQQPGMA\
VDLYRPDDYYDILFPGVQTFVHSVQYLDPRHWGPTLFNAISQAFWRVIQNDIPRLTSQEL\
ERRTQRYLRDSLARFLEETTWTVINAPVNWYNSLQDYYSTLSPIRPTMVRQVANREGLQI\
SFGHTYDNIDEADSIQQVTERWEAQSQSPNVQSGEFIEKFEAPGGANQRTAPQWMLPLLL\
GLYGSVTSALKAYEDGPNKKKRKLSRGSSQKTKGTSASAKARHKRRNRSSRS"
)
torch.manual_seed(1509)
np.random.seed(1509)
# protein = ESMProtein(
#     sequence=seqss[120][1]
# )  

full_res = {}

with torch.no_grad():
    for i, seq in zip(range(len(seqss)), seqss):
        # protein = protein = ESMProtein(sequence=seqss[120][3])
        protein = model.generate(protein, GenerationConfig(track="secondary_structure", num_steps=16))
        protein = model.generate(protein, GenerationConfig(track="sasa", num_steps=16))
        protein = model.generate(protein, GenerationConfig(track="structure", num_steps=16))
        res = model.encode(protein)
        # data = {}
        # data["randomseed"] = 1509 
        # data["model"] = "esm3_sm_open_v1"
        # data["id"] = seq[0]
        # data["ori_seq"] = seq[1]  
        # data["seq_t"] = res.sequence.cpu().numpy() 
        # data["strcture_t"] = res.structure.cpu().numpy() 
        # data["second_t"] = res.secondary_structure.cpu().numpy()
        # data["sasa_t"] = res.sasa.cpu().numpy() 
        # data["coordinates"] = res.coordinates.cpu().numpy() 
        # full_res.append()
        print(res)
        break
    # protein = model.generate(protein, GenerationConfig(track="secondary_structure", num_steps=16))
    # protein = model.generate(protein, GenerationConfig(track="sasa", num_steps=16))
    # protein = model.generate(protein, GenerationConfig(track="structure", num_steps=16))
    # protein = model.generate(protein, GenerationConfig(track="secondary_structure", num_steps=16))
    # protein = model.generate(protein, GenerationConfig(track="sasa", num_steps=16))
    # protein = model.generate(protein, GenerationConfig(track="structure", num_steps=16))

ESMProteinTensor(sequence=tensor([ 0, 20,  6,  5,  5,  4, 11,  4,  4,  6, 13,  4, 12,  5, 11,  7,  8,  9,
         5,  5,  5,  5, 11,  6, 18,  8,  7,  5,  9, 12,  5,  5,  6,  9,  5,  5,
         5,  5, 12,  9,  7, 16,  4,  5,  8,  7,  5, 11,  7,  9,  6,  4, 11, 11,
         8,  9,  5, 12,  5,  5, 12,  6,  4, 12, 14, 16,  5, 19,  5,  7, 12,  8,
         6,  5, 14,  5,  5, 12,  5,  6, 18,  5,  5,  4,  4, 16, 11,  7, 11,  6,
         7,  8,  5,  7,  5, 16,  7,  6, 19, 10, 18, 18,  8, 13, 22, 13, 21, 15,
         7,  8, 11,  7,  6,  4, 19, 16, 16, 14,  6, 20,  5,  7, 13,  4, 19, 10,
        14, 13, 13, 19, 19, 13, 12,  4, 18, 14,  6,  7, 16, 11, 18,  7, 21,  8,
         7, 16, 19,  4, 13, 14, 10, 21, 22,  6, 14, 11,  4, 18, 17,  5, 12,  8,
        16,  5, 18, 22, 10,  7, 12, 16, 17, 13, 12, 14, 10,  4, 11,  8, 16,  9,
         4,  9, 10, 10, 11, 16, 10, 19,  4, 10, 13,  8,  4,  5, 10, 18,  4,  9,
         9, 11, 11, 22, 11,  7, 12, 17,  5, 14,  7, 17, 22, 19, 17,  8,  4, 16,
        13, 19

In [3]:
import pickle 
with open("/home/tyfei/datasets/ion_channel/Interprot/ion_channel/mmseqs_kingdom/Archaea.pkl", "rb") as f:
    datapos = pickle.load(f)
    for i in datapos:
        i["structure_t"] = i.pop("strcture_t")
 
with open("/home/tyfei/datasets/ion_channel/Interprot/Negative_sample/virus_proteins_TMHMM_keywords.pkl", "rb") as f:
    dataneg = pickle.load(f)
    for i in dataneg:
        i["structure_t"] = i.pop("strcture_t")
    dataneg = dataneg[:len(datapos)]
datapos[0], dataneg[0]

({'randomseed': 1509,
  'model': 'esm3_sm_open_v1',
  'id': 'A0A533WCC2',
  'ori_seq': 'MAIVHKEAQPPSWYGSVRALFLKIFTRVQQDQPRRILISGKPSPPDTFKLNSKAIFLAGAAAIFVLSLLQTPVHAQAATSPNEVISSTSAKFIAAGVVFALAAVGAGYGVGHAGAAAMAALAERPEVRSMALIITALAEAIAIYGLVMAIIILGTAST',
  'seq_t': array([ 0, 20,  5, 12,  7, 21, 15,  9,  5, 16, 14, 14,  8, 22, 19,  6,  8,
          7, 10,  5,  4, 18,  4, 15, 12, 18, 11, 10,  7, 16, 16, 13, 16, 14,
         10, 10, 12,  4, 12,  8,  6, 15, 14,  8, 14, 14, 13, 11, 18, 15,  4,
         17,  8, 15,  5, 12, 18,  4,  5,  6,  5,  5,  5, 12, 18,  7,  4,  8,
          4,  4, 16, 11, 14,  7, 21,  5, 16,  5,  5, 11,  8, 14, 17,  9,  7,
         12,  8,  8, 11,  8,  5, 15, 18, 12,  5,  5,  6,  7,  7, 18,  5,  4,
          5,  5,  7,  6,  5,  6, 19,  6,  7,  6, 21,  5,  6,  5,  5,  5, 20,
          5,  5,  4,  5,  9, 10, 14,  9,  7, 10,  8, 20,  5,  4, 12, 12, 11,
          5,  4,  5,  9,  5, 12,  5, 12, 19,  6,  4,  7, 20,  5, 12, 12, 12,
          4,  6, 11,  5,  8, 11,  2]),
  'second_

In [4]:
len(datapos[0]["structure_t"]), len(datapos[0]["seq_t"]), len(datapos[0]["ori_seq"])

(160, 160, 158)

In [5]:
data = [] 
data.extend(datapos)
data.extend(dataneg) 
label = [1]*len(datapos)+[0]*len(dataneg)

In [6]:
import VirusDataset 
import random 
aug = VirusDataset.DataAugmentation([2,4], [(0.1, -1), (0.1, 0.2)], [-1.0, 0.4], [75, 100, 120, 130])
ts = random.sample(range(len(data)), 100) 
ds = VirusDataset.ESM3MultiTrackDataset([data[i] for i in ts], [data[i] for i in ts], [label[i] for i in ts], augment=aug)
dl = VirusDataset.ESM3datamodule(ds, ds)
# for i in enumerate(dl):
#     print(i)
#     break

In [1]:
import pytorch_lightning as L 

trainer = L.Trainer(accelerator="gpu", devices=[7], accumulate_grad_batches=12, max_epochs=7)


/home/tyfei/anaconda3/envs/esm3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [4]:
trainer.train_dataloader

In [9]:
import models 
import torch.nn as nn 
import torch
import importlib 
importlib.reload(models)
newmodel = models.IonclfBaseline()
newmodel = newmodel#.cuda(6)
t = data[0]["seq_t"]
t = torch.tensor(t)
embed = nn.Embedding(35, 16)
print(t)
t = t.unsqueeze(0)
embed(t)
newmodel({"seq_t":t})
# newmodel = models.IonclfESM3(model, addadversial=False)#.cuda(6)
# with torch.no_grad():
#     q = newmodel._common_training_step(({"sequence_t":res.sequence.unsqueeze(0)}, torch.tensor([1]).cuda(6), {"sequence_t":res.sequence.unsqueeze(0)}))
#     #q = newmodel({"sequence_t":res.sequence.unsqueeze(0)})
# q 

tensor([ 0, 20,  5, 12,  7, 21, 15,  9,  5, 16, 14, 14,  8, 22, 19,  6,  8,  7,
        10,  5,  4, 18,  4, 15, 12, 18, 11, 10,  7, 16, 16, 13, 16, 14, 10, 10,
        12,  4, 12,  8,  6, 15, 14,  8, 14, 14, 13, 11, 18, 15,  4, 17,  8, 15,
         5, 12, 18,  4,  5,  6,  5,  5,  5, 12, 18,  7,  4,  8,  4,  4, 16, 11,
        14,  7, 21,  5, 16,  5,  5, 11,  8, 14, 17,  9,  7, 12,  8,  8, 11,  8,
         5, 15, 18, 12,  5,  5,  6,  7,  7, 18,  5,  4,  5,  5,  7,  6,  5,  6,
        19,  6,  7,  6, 21,  5,  6,  5,  5,  5, 20,  5,  5,  4,  5,  9, 10, 14,
         9,  7, 10,  8, 20,  5,  4, 12, 12, 11,  5,  4,  5,  9,  5, 12,  5, 12,
        19,  6,  4,  7, 20,  5, 12, 12, 12,  4,  6, 11,  5,  8, 11,  2])


(tensor([[0.5676]], grad_fn=<SigmoidBackward0>),
 tensor([[0.5179]], grad_fn=<SigmoidBackward0>))

In [3]:
for i, j in newmodel.named_parameters():
    print(i)

feature_extract.embed.weight
feature_extract.inc.weight
feature_extract.inc.bias
feature_extract.sa1.mha.in_proj_weight
feature_extract.sa1.mha.in_proj_bias
feature_extract.sa1.mha.out_proj.weight
feature_extract.sa1.mha.out_proj.bias
feature_extract.sa1.ln.weight
feature_extract.sa1.ln.bias
feature_extract.sa1.ff_self.0.weight
feature_extract.sa1.ff_self.0.bias
feature_extract.sa1.ff_self.1.weight
feature_extract.sa1.ff_self.1.bias
feature_extract.sa1.ff_self.3.weight
feature_extract.sa1.ff_self.3.bias
feature_extract.sa2.mha.in_proj_weight
feature_extract.sa2.mha.in_proj_bias
feature_extract.sa2.mha.out_proj.weight
feature_extract.sa2.mha.out_proj.bias
feature_extract.sa2.ln.weight
feature_extract.sa2.ln.bias
feature_extract.sa2.ff_self.0.weight
feature_extract.sa2.ff_self.0.bias
feature_extract.sa2.ff_self.1.weight
feature_extract.sa2.ff_self.1.bias
feature_extract.sa2.ff_self.3.weight
feature_extract.sa2.ff_self.3.bias
feature_extract.sa3.mha.in_proj_weight
feature_extract.sa3.mha.

In [55]:
for i, j in newmodel.named_parameters():
    if "esm_model" in i:
        j.requires_grad = False
        if "esm_model.transformer" in i:
            if "47" in i or "46" in i or "45" in i:
                j.requires_grad = True 
    if j.requires_grad:
        print(i)

esm_model.transformer.blocks.45.attn.layernorm_qkv.0.weight
esm_model.transformer.blocks.45.attn.layernorm_qkv.0.bias
esm_model.transformer.blocks.45.attn.layernorm_qkv.1.weight
esm_model.transformer.blocks.45.attn.out_proj.weight
esm_model.transformer.blocks.45.attn.q_ln.weight
esm_model.transformer.blocks.45.attn.k_ln.weight
esm_model.transformer.blocks.45.ffn.0.weight
esm_model.transformer.blocks.45.ffn.0.bias
esm_model.transformer.blocks.45.ffn.1.weight
esm_model.transformer.blocks.45.ffn.3.weight
esm_model.transformer.blocks.46.attn.layernorm_qkv.0.weight
esm_model.transformer.blocks.46.attn.layernorm_qkv.0.bias
esm_model.transformer.blocks.46.attn.layernorm_qkv.1.weight
esm_model.transformer.blocks.46.attn.out_proj.weight
esm_model.transformer.blocks.46.attn.q_ln.weight
esm_model.transformer.blocks.46.attn.k_ln.weight
esm_model.transformer.blocks.46.ffn.0.weight
esm_model.transformer.blocks.46.ffn.0.bias
esm_model.transformer.blocks.46.ffn.1.weight
esm_model.transformer.blocks.46

In [10]:
trainer.fit(newmodel, dl)

You are using a CUDA device ('NVIDIA H800 PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | acc             | BinaryAccuracy   | 0      | train
1 | feature_extract | SeqTransformer   | 131 K  | train
2 | reverse         | GradientReversal | 0      | train
3 | clf             | Linearcls        | 2.7 K  | train
4 | dis             | Linearcls        | 2.7 K  | train
-------------------------------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.547     Total estimated model params size (

Sanity Checking: |          | 0/? [00:00<?, ?it/s]get val loader
get train loader                                                           
Epoch 6: 100%|██████████| 85/85 [00:03<00:00, 21.51it/s, v_num=55, predict loss=1.130, adversial loss=0.826, acc=0.000]

`Trainer.fit` stopped: `max_epochs=7` reached.


Epoch 6: 100%|██████████| 85/85 [00:03<00:00, 21.36it/s, v_num=55, predict loss=1.130, adversial loss=0.826, acc=0.000]


In [49]:
t = [] 
for i in range(100):
    t.append(random.random())
np.mean(t)

0.5063902083385337

In [117]:
len(newmodel.training_step_outputs)

0

In [89]:
# test += 1 
# protein.to_pdb("./structure_temp/test_%i.pdb"%test)
import py3Dmol 
with open("./structure_temp/test_%i.pdb"%3) as ifile:
    system = "".join([x for x in ifile])
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(system)
view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.zoomTo()
view.show()

In [18]:
test += 1 
protein.to_pdb("./structure_temp/test_%i.pdb"%test)
import py3Dmol 
with open("./structure_temp/test_%i.pdb"%test) as ifile:
    system = "".join([x for x in ifile])
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(system)
view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [15]:
res.secondary_structure

In [9]:
protein.__dir__()

['__module__',
 '__annotations__',
 '__len__',
 'from_pdb',
 'from_protein_chain',
 'to_pdb',
 'to_pdb_string',
 'to_protein_chain',
 '__doc__',
 '__attrs_attrs__',
 '__getstate__',
 '__setstate__',
 '__repr__',
 '__eq__',
 '__ne__',
 '__hash__',
 '__init__',
 '__match_args__',
 '__attrs_own_setattr__',
 '__slots__',
 'coordinates',
 'function_annotations',
 'plddt',
 'ptm',
 'sasa',
 'secondary_structure',
 'sequence',
 '__weakref__',
 '__new__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__gt__',
 '__ge__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [10]:
protein.s 

In [2]:
profileseqs = [i for i in seqs]

In [3]:
profileseqs

[('A0A076N198',
  'MLLRLLRTTDHKQIGIMYLVTSFAFFMVGGAMAMLIRTELARPGQQFLSQEQYNQLFTMHGTIMLLYATPILRIAQHVEGAENGRLRRRRGRMVANVGDFEVLRALRRIGPPYAMTPSQLRAEMLISSSGLTGRLHRLERDGWIMREPRPTTSAASWSP'),
 ('A0A0P0H782',
  'WSTIHKDIGTLYFIFGASAGMVGTSLSLLIRAEFGQPGYLIGEDQIYNVLYSSCIRIGGFGNCYSFMLGAPMYIPSNMYTSGYYLLPYPFTKYLKSVLVLERFTPLYHQVLPCSSISDLQSSHYICSGIIYFRSSKFYYNNYQYTSSWYIIGSNTFICWAVAITALLLLLSLPVQAGAITMLQLSKQMLHSSIRQEGEIQFYINISFVFWSPWKF'),
 ('A0A0S2RSV8',
  'HFLGLAGMPRRYSDYPDNFTSWNIISSFGSYISLFSMMMMMIIIWESFVNQRIILFPLNMPTSIEWFQNLPPSEHSYNELPILSLLMWQIMCNGFKPHL'),
 ('A0A1C9EIW6',
  'MLVRWLFSTNHKDIGVLYVLVSMWSGVLGLVMSVLIRMELAXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXLANSCVDVVLHDTYYVVAHFHYVLSMGAVFAIFGGFVTWFSVVTGIGLNQKLLKIHFFVMFVGVNLTFFPQHFMGLSGMPRRY